<a href="https://colab.research.google.com/github/jonG312/cuda_ejercicios/blob/main/cuda_ejercicio_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!python --version
!nvcc --version
!pip install nvcc4jupyter
%load_ext nvcc4jupyter

In [ ]:
!apt install nvidia-cuda-toolkit


In [ ]:
!python --version
!nvcc --version

In [6]:
%%cuda

#include <stdio.h>
#include <iostream>
#include <math.h>
#include <cuda_runtime.h>

using namespace std;

// Definición de la función floydKernel
__global__ void floydKernel(float *C, int k, int N) {
    // Implementación de floydKernel
}

//**************************************************************************
int main(int argc, char *argv[])
//**************************************************************************
{
    int Bsize, NBlocks;
    if (argc != 3)
    {
        cout << "Uso: transformacion Num_bloques Tam_bloque  " << endl;
        return (0);
    }
    else
    {
        NBlocks = atoi(argv[1]);
        Bsize = atoi(argv[2]);
    }

    const int N = Bsize * NBlocks;
    float *A, *B, *C, *D;
    A = new float[N];
    B = new float[N];
    C = new float[N];
    D = new float[NBlocks];
    float mx;

    // Inicializar arrays A y B
    for (int i = 0; i < N; i++)
    {
        A[i] = (float)(1.5 * (1 + (5 * i) % 7) / (1 + i % 5));
        B[i] = (float)(2.0 * (2 + i % 5) / (1 + i % 7));
    }

    // Declarar y asignar memoria en el device
    float *d_A, *d_B, *d_C, *d_D;
    cudaMalloc(&d_A, N * sizeof(float));
    cudaMalloc(&d_B, N * sizeof(float));
    cudaMalloc(&d_C, N * sizeof(float));
    cudaMalloc(&d_D, NBlocks * sizeof(float));

    // Copiar datos del host al device
    cudaMemcpy(d_A, A, N * sizeof(float), cudaMemcpyHostToDevice);
    cudaMemcpy(d_B, B, N * sizeof(float), cudaMemcpyHostToDevice);

    // Configurar dimensiones de bloque y grid
    dim3 blockSize(Bsize, Bsize);
    dim3 gridSize((N + blockSize.x - 1) / blockSize.x, (N + blockSize.y - 1) / blockSize.y);

    // Iniciar temporizador
    double t1 = clock();

    // Ejecutar kernel en el device
    for (int k = 0; k < N; k++)
    {
        floydKernel<<<gridSize, blockSize>>>(d_C, k, N);
        cudaDeviceSynchronize();
    }

    // Detener temporizador
    double t2 = clock();

    // Calcular tiempo de ejecución
    double gpu_execution_time = (t2 - t1) / CLOCKS_PER_SEC;

    // Copiar resultados de vuelta al host
    cudaMemcpy(C, d_C, N * sizeof(float), cudaMemcpyDeviceToHost);

    // Imprimir resultados
    cout << "Resultados:" << endl;
    for (int k = 0; k < NBlocks; k++)
    {
        cout << "D[" << k << "] = " << D[k] << endl;
    }
    cout << "El valor máximo en C es: " << mx << endl;
    cout << "Tiempo de ejecución en GPU: " << gpu_execution_time << " segundos" << endl;

    // Liberar memoria en el device
    cudaFree(d_A);
    cudaFree(d_B);
    cudaFree(d_C);
    cudaFree(d_D);

    // Liberar memoria en el host
    delete[] A;
    delete[] B;
    delete[] C;
    delete[] D;

    return 0;
}


Uso: transformacion Num_bloques Tam_bloque  



In [16]:
%%cuda
#include <stdio.h>
#include <stdlib.h>
#include <iostream>
#include <math.h>
#include <chrono>

using namespace std;

// Definir tamaño máximo de la matriz
#define MAX_SIZE 2000

// Kernel para el algoritmo de Floyd en CUDA
__global__ void floydKernel(float* matrix, int k, int size) {
    int i = blockIdx.y * blockDim.y + threadIdx.y;
    int j = blockIdx.x * blockDim.x + threadIdx.x;

    if (i < size && j < size) {
        if (i != j && i != k && j != k) {
            float ikj = matrix[i * size + k] + matrix[k * size + j];
            if (ikj < matrix[i * size + j]) {
                matrix[i * size + j] = ikj;
            }
        }
    }
}

// Función para ejecutar algoritmo de Floyd en la CPU
void floydCPU(float* matrix, int size) {
    for (int k = 0; k < size; ++k) {
        for (int i = 0; i < size; ++i) {
            for (int j = 0; j < size; ++j) {
                if (i != j && i != k && j != k) {
                    float ikj = matrix[i * size + k] + matrix[k * size + j];
                    if (ikj < matrix[i * size + j]) {
                        matrix[i * size + j] = ikj;
                    }
                }
            }
        }
    }
}

int main() {
    // Tamaños de problema y bloques
    int N_values[] = {400, 1000, 1400, 2000};
    int Bsize_values[] = {8, 16, 32};

    // Iterar sobre diferentes tamaños de problema y bloques
    for (int i = 0; i < sizeof(N_values) / sizeof(N_values[0]); ++i) {
        for (int j = 0; j < sizeof(Bsize_values) / sizeof(Bsize_values[0]); ++j) {
            int N = N_values[i];
            int Bsize = Bsize_values[j];
            int NBlocks = (N + Bsize - 1) / Bsize;

            // Declarar matrices en el host
            float *h_matrix;
            h_matrix = new float[N * N];

            // Inicializar matriz en el host (para este ejemplo, se inicializa con valores aleatorios)
            for (int i = 0; i < N * N; i++) {
                h_matrix[i] = (float)(rand() % 100); // Valores aleatorios entre 0 y 99
            }

            // Declarar matriz en el device
            float *d_matrix;
            cudaMalloc(&d_matrix, N * N * sizeof(float));

            // Copiar matriz del host al device
            cudaMemcpy(d_matrix, h_matrix, N * N * sizeof(float), cudaMemcpyHostToDevice);

            // Configurar tamaños de bloque y grid
            dim3 blockSize(Bsize, Bsize);
            dim3 gridSize((N + blockSize.x - 1) / blockSize.x, (N + blockSize.y - 1) / blockSize.y);

            // Medir tiempo de ejecución en GPU
            auto start_time_gpu = chrono::high_resolution_clock::now();

            // Ejecutar kernel para algoritmo de Floyd en la GPU
            for (int k = 0; k < N; k++) {
                floydKernel<<<gridSize, blockSize>>>(d_matrix, k, N);
                cudaDeviceSynchronize();
            }

            auto end_time_gpu = chrono::high_resolution_clock::now();
            chrono::duration<double> gpu_execution_time = end_time_gpu - start_time_gpu;

            // Copiar matriz resultado de vuelta al host
            cudaMemcpy(h_matrix, d_matrix, N * N * sizeof(float), cudaMemcpyDeviceToHost);

            // Medir tiempo de ejecución en CPU
            auto start_time_cpu = chrono::high_resolution_clock::now();

            // Ejecutar algoritmo de Floyd en la CPU
            floydCPU(h_matrix, N);

            auto end_time_cpu = chrono::high_resolution_clock::now();
            chrono::duration<double> cpu_execution_time = end_time_cpu - start_time_cpu;

            // Imprimir tiempos de ejecución y ganancia de velocidad
            cout << "N = " << N << ", Bsize = " << Bsize << endl;
            cout << "TCPU(sec): " << cpu_execution_time.count() << endl;
            cout << "TGPU1D(sec): " << gpu_execution_time.count() << endl;
            cout << "SGPU1D: " << cpu_execution_time.count() / gpu_execution_time.count() << endl;

            // Liberar memoria en el device
            cudaFree(d_matrix);

            // Liberar memoria en el host
            delete[] h_matrix;

            cout << endl;
        }
    }

    return 0;
}


N = 400, Bsize = 8
TCPU(sec): 0.334253
TGPU1D(sec): 0.0454335
SGPU1D: 7.35697

N = 400, Bsize = 16
TCPU(sec): 0.327732
TGPU1D(sec): 0.00529897
SGPU1D: 61.8482

N = 400, Bsize = 32
TCPU(sec): 0.33628
TGPU1D(sec): 0.00601956
SGPU1D: 55.8646

N = 1000, Bsize = 8
TCPU(sec): 6.19635
TGPU1D(sec): 0.0472916
SGPU1D: 131.024

N = 1000, Bsize = 16
TCPU(sec): 5.37546
TGPU1D(sec): 0.0354677
SGPU1D: 151.559

N = 1000, Bsize = 32
TCPU(sec): 6.09969
TGPU1D(sec): 0.0443184
SGPU1D: 137.633

N = 1400, Bsize = 8
TCPU(sec): 15.4307
TGPU1D(sec): 0.121424
SGPU1D: 127.08

N = 1400, Bsize = 16
TCPU(sec): 15.3478
TGPU1D(sec): 0.103267
SGPU1D: 148.623

N = 1400, Bsize = 32
TCPU(sec): 15.8746
TGPU1D(sec): 0.128572
SGPU1D: 123.468

N = 2000, Bsize = 8
TCPU(sec): 45.9709
TGPU1D(sec): 0.278711
SGPU1D: 164.941

N = 2000, Bsize = 16
TCPU(sec): 46.693
TGPU1D(sec): 0.263058
SGPU1D: 177.501

N = 2000, Bsize = 32
TCPU(sec): 46.7352
TGPU1D(sec): 0.321709
SGPU1D: 145.272


